In [55]:
import pandas as pd

In [56]:
df_copy = pd.read_csv("diabetic_data.csv")

In [57]:
df = df_copy.copy()

In [ ]:
# Get number of features (columns)
num_features = df.shape[1]

print(f"Number of features: {num_features}")

In [58]:
X = df[['race', 'gender', 'age','admission_type_id','admission_source_id', 'time_in_hospital', 'medical_specialty',
        'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3','metformin', 'glipizide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'insulin', 'change', 'readmitted']]

Preprocessing

In [ ]:
X.admission_type_id.value_counts()

Checked the mapping of "admission_type_id" and cleaned accordingly.

In [ ]:
# Remove rows where 'admission_type_id' is 6 or 8
X = X[~X['admission_type_id'].isin([6, 8])]

In [ ]:
X.admission_type_id.value_counts()

In [ ]:
# replace the "7" label of datapoints with "6" label
X['admission_type_id'] = X['admission_type_id'].replace(7, 6)

In [ ]:
X.admission_type_id.value_counts().sort_index()

Checked the mapping of "admission_source_id" and cleaned accordingly.

In [ ]:
X.admission_source_id.value_counts().sort_index()

In [ ]:
# Get valid labels 
X = X[X['admission_source_id'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 22, 25])]

In [ ]:
# replace the "13" label of datapoints with "12" label
X['admission_source_id'] = X['admission_source_id'].replace(13, 12)

# replace the "14" label of datapoints with "13" label
X['admission_source_id'] = X['admission_source_id'].replace(14, 13)

# replace the "22" label of datapoints with "14" label
X['admission_source_id'] = X['admission_source_id'].replace(22, 14)

# replace the "25" label of datapoints with "15" label
X['admission_source_id'] = X['admission_source_id'].replace(25, 15)

In [ ]:
X.shape

In [59]:
X.medical_specialty.value_counts()

medical_specialty
?                         49949
InternalMedicine          14635
Emergency/Trauma           7565
Family/GeneralPractice     7440
Cardiology                 5352
                          ...  
Proctology                    1
Speech                        1
SportsMedicine                1
Perinatology                  1
Neurophysiology               1
Name: count, Length: 73, dtype: int64

In [60]:
X = X.drop(columns=['medical_specialty']) # has 49% '?', therefore dropped the column

In [61]:
X.gender.value_counts()

gender
Female             54708
Male               47055
Unknown/Invalid        3
Name: count, dtype: int64

In [62]:
# Identify rows where 'race' is '?'
rows_to_drop = X[X['race'] == '?'].index
# Drop these rows
X.drop(rows_to_drop, inplace=True)

In [63]:
# Identify rows where 'gender' is 'invalid'
rows_to_drop = X[X['gender'] == 'Unknown/Invalid'].index
# Drop these rows
X.drop(rows_to_drop, inplace=True)

In [64]:
X.diag_1.unique().shape

(715,)

In [65]:
# Identify rows where 'diag_1' is '?'
rows_to_drop = X[X['diag_1'] == '?'].index
# Drop these rows
X.drop(rows_to_drop, inplace=True)

In [66]:
# Identify rows where 'diag_2' is '?'
rows_to_drop = X[X['diag_2'] == '?'].index
# Drop these rows
X.drop(rows_to_drop, inplace=True)

In [67]:
# Identify rows where 'diag_3' is '?'
rows_to_drop = X[X['diag_3'] == '?'].index
# Drop these rows
X.drop(rows_to_drop, inplace=True)

In [68]:
import pandas as pd

# Define the mapping for ICD9 codes to group names
def map_icd9(code):
    try:
        code = str(code)
        if code.startswith("250"):
            return "Diabetes"
        elif 390 <= int(code) <= 459 or code == "785":
            return "Circulatory"
        elif 460 <= int(code) <= 519 or code == "786":
            return "Respiratory"
        elif 520 <= int(code) <= 579 or code == "787":
            return "Digestive"
        elif 800 <= int(code) <= 999:
            return "Injury"
        elif 710 <= int(code) <= 739:
            return "Musculoskeletal"
        elif 580 <= int(code) <= 629 or code == "788":
            return "Genitourinary"
        elif 140 <= int(code) <= 239:
            return "Neoplasms"
        else:
            return "Other"
    except ValueError:
        return "Other"

# Apply the mapping
X['diag_1'] = X['diag_1'].apply(map_icd9)
X['diag_2'] = X['diag_2'].apply(map_icd9)
X['diag_3'] = X['diag_3'].apply(map_icd9)


In [69]:
X['readmitted'] = X['readmitted'].map({
    '>30': 0,
    'NO': 0,
    '<30': 1
})

In [70]:
X.columns

Index(['race', 'gender', 'age', 'admission_type_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'metformin',
       'glipizide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'insulin',
       'change', 'readmitted'],
      dtype='object')

In [71]:
X['readmitted'].value_counts()

readmitted
0    86986
1    11066
Name: count, dtype: int64

Encoding

Ordinal encoding for age

In [72]:
from sklearn.preprocessing import OrdinalEncoder

# Define the order of categories
categories_order = [
    '[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)',
    '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'
]

# Apply Ordinal Encoding
encoder = OrdinalEncoder(categories=[categories_order])
X['age'] = encoder.fit_transform(X[['age']])

In [73]:
X.head()

,race,gender,age,admission_type_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,diag_2,diag_3,metformin,glipizide,pioglitazone,rosiglitazone,acarbose,insulin,change,readmitted
1,Caucasian,Female,1.0,1,7,3,59,0,18,0,...,Diabetes,Other,No,No,No,No,No,Up,Ch,0
2,AfricanAmerican,Female,2.0,1,7,2,11,5,13,2,...,Diabetes,Other,No,Steady,No,No,No,No,No,0
3,Caucasian,Male,3.0,1,7,2,44,1,16,0,...,Diabetes,Circulatory,No,No,No,No,No,Up,Ch,0
4,Caucasian,Male,4.0,1,7,1,51,0,8,0,...,Neoplasms,Diabetes,No,Steady,No,No,No,Steady,Ch,0
5,Caucasian,Male,5.0,2,2,3,31,6,16,0,...,Circulatory,Diabetes,No,No,No,No,No,Steady,No,0


Ordinal encoding for medicine, maybe categorical(needs checking)

In [74]:
# Define columns to encode
columns_to_encode = ['metformin', 'glipizide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'insulin']

# Define the order of categories
category_order = ['No', 'Down', 'Steady', 'Up']

# Initialize OrdinalEncoder with the specified order
encoder = OrdinalEncoder(categories=[category_order] * len(columns_to_encode))

# Apply the encoder only to the specified columns
encoded_values = encoder.fit_transform(X[columns_to_encode])

# Replace original columns with encoded values
X[columns_to_encode] = encoded_values

In [ ]:
X.number_inpatient.value_counts()

One-hot encoding for categorical variables

In [75]:
df_encoded = pd.get_dummies(X, columns=['admission_type_id','admission_source_id', 'race', 'gender', 'change', 'diag_1', 'diag_2', 'diag_3'])

Checking the dataset after encoding

In [76]:
df_encoded.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,metformin,glipizide,...,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Genitourinary,diag_3_Injury,diag_3_Musculoskeletal,diag_3_Neoplasms,diag_3_Other,diag_3_Respiratory
1,1.0,3,59,0,18,0,0,0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,2.0,2,11,5,13,2,0,1,0.0,2.0,...,False,False,False,False,False,False,False,False,True,False
3,3.0,2,44,1,16,0,0,0,0.0,0.0,...,False,True,False,False,False,False,False,False,False,False
4,4.0,1,51,0,8,0,0,0,0.0,2.0,...,False,False,True,False,False,False,False,False,False,False
5,5.0,3,31,6,16,0,0,0,0.0,0.0,...,False,False,True,False,False,False,False,False,False,False


In [77]:
df_encoded.columns.shape

(76,)

Use scaling if necessary, and also fix class imbalance issue

In [80]:
X.select_dtypes(include='int').columns

Index(['admission_type_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'readmitted'],
      dtype='object')

In [81]:
X.select_dtypes(include='float').columns

Index(['age', 'metformin', 'glipizide', 'pioglitazone', 'rosiglitazone',
       'acarbose', 'insulin'],
      dtype='object')

In [82]:
X.select_dtypes(include='object').columns

Index(['race', 'gender', 'diag_1', 'diag_2', 'diag_3', 'change'], dtype='object')

In [ ]:
df.to_csv('filename.csv', index=False)

In [ ]:
df_encoded.columns

In [ ]:
print(df_encoded.describe())

In [ ]:
# Select only numerical features
numerical_features = df_encoded.select_dtypes(include=['number']).columns
print("Numerical Features:", numerical_features)

In [ ]:
# Get number of features (columns)
numof_features = df_encoded.shape[1]

print(f"Number of features: {numof_features}")

In [ ]:
df_encoded.readmitted.value_counts()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from collections import Counter



X = df_encoded.drop(columns=['readmitted'])  # Independent variables
y = df_encoded['readmitted']                 # Target variable

# Step 2: Handle Class Imbalance using SMOTE-ENN
print("Before SMOTE-ENN:", Counter(y))
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
print("After SMOTE-ENN:", Counter(y_resampled))

# Split the resampled data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 3: Train Random Forest Classifier with Hyperparameters
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=12,              # Increased tree depth
    min_samples_split=5,       # Minimum samples required to split a node
    min_samples_leaf=5,        # Minimum samples required in each leaf
    max_features='sqrt',
    class_weight='balanced',   # Adjusted class weights
    random_state=42,
    n_jobs=-1
)

# Fit the model
rf.fit(X_train, y_train)

# Step 4: Evaluate the Model
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]  # Probabilities for ROC-AUC

print("Classification Report:")
print(classification_report(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Calculate Accuracy for Training and Testing
train_accuracy = accuracy_score(y_train, rf.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")

# Calculate Recall for Class 1
recall = recall_score(y_test, y_pred)
print(f"Recall for Class 1 (Readmission): {recall:.4f}")


In [ ]:
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from collections import Counter

# Assuming X and y are your original dataset features and target variables
print("Before SMOTE-ENN:", Counter(y))

# Apply SMOTE-ENN for balancing the dataset
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X, y)

print("After SMOTE-ENN:", Counter(y_resampled))

# Split the resampled data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 1: Scaling the features for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 2: Train Logistic Regression Model
log_reg = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Step 3: Evaluate the Logistic Regression Model
y_pred = log_reg.predict(X_test_scaled)
y_pred_proba = log_reg.predict_proba(X_test_scaled)[:, 1]  # Probabilities for ROC-AUC

# Classification Report
print("Classification Report for Logistic Regression:")
print(classification_report(y_test, y_pred))

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Training and Testing Accuracy
train_accuracy = log_reg.score(X_train_scaled, y_train)
test_accuracy = log_reg.score(X_test_scaled, y_test)
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")


In [ ]:
# Install XGBoost if not already installed
!pip install xgboost

# Import necessary libraries
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from collections import Counter

X = df_encoded.drop(columns=['readmitted'])  # Independent variables
y = df_encoded['readmitted']                 # Target variable
# Assuming X and y are your original dataset features and target variables
print("Before SMOTE-ENN:", Counter(y))

# Apply SMOTE-ENN for balancing the dataset
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X, y)

print("After SMOTE-ENN:", Counter(y_resampled))

# Split the resampled data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize XGBoost model
xgb_model = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')

# Train the XGBoost model
xgb_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = xgb_model.predict(X_test_scaled)
y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for ROC-AUC

# Evaluate the model
print("\nXGBoost Classification Report:")
print(classification_report(y_test, y_pred))
print(f"XGBoost ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
